In [2]:
import pandas as pd

df = pd.read_csv("./data/Trade_DetailedTradeMatrix.csv", encoding='latin-1')

This is the part where we analyze the data.

In [67]:
import duckdb

## This function calculates the dependency of a country on another country for a certain good
## For example: using dependency('United Arab Emirates', 2019, 'Rice') will give you the top
#

def dependency(country, year, item):
    query = '''
    select Partner_Countries,
        sum(Value) /
        (
            select
                sum(Value)
            from df
            where Year = {year}
            and Reporter_Countries = '{country}'
            and Item = '{item}'
            and Element = 'Import Quantity'
        ) as dep_ratio
    from df
    where Year = {year}
    and Reporter_Countries = '{country}'
    and Element = 'Import Quantity'
    and Item = '{item}'
    and Value > 0
    group by Partner_Countries
    order by dep_ratio desc; 
'''.format(item = item, country = country, year = year)
    
    return duckdb.sql(query).fetchdf()

dependency('United Arab Emirates', 2019, 'Rice')

,Partner_Countries,dep_ratio
0,Pakistan,0.573980
1,India,0.364404
2,United States of America,0.026558
3,Thailand,0.011904
4,Philippines,0.007916
5,Sri Lanka,0.007144
6,Italy,0.003958
7,United Kingdom of Great Britain and Northern I...,0.002533
8,Belgium,0.000821
9,"China, mainland",0.000317


In [82]:
def product_leverage(country, year, item):
    query = '''
        drop table if exists exp;
        drop table if exists imp;

        create table exp as
        select Reporter_Countries as reporting,
                    sum(Value) as trade
                from df
                where Year = {year}
                and Partner_Countries = '{country}'
                and Element = 'Import Quantity'
                and Item = '{item}'
                and Value > 0
                group by Reporter_Countries
                order by trade desc;

        create table imp as
        select Reporter_Countries,
                    sum(Value) as import_quantity
                from df
                where Year = {year}
                and Element = 'Import Quantity'
                and Item = '{item}'
                and Value > 0
                group by Reporter_Countries
                order by import_quantity desc;

        select Reporter_Countries, trade, import_quantity, trade / import_quantity as dep
        from exp
        left join imp
        on imp.Reporter_Countries = exp.reporting
        where dep between 0 and 1
        order by dep desc;
        '''.format(year = year, country = country, item = item)
    out = duckdb.query(query)

    return out.fetchdf()

product_leverage('India', 2019, 'Rice')

┌──────────────────────────────────────────────────────┬──────────┬────────────────────┬────────────────────────┐
│                  Reporter_Countries                  │  trade   │  import_quantity   │          dep           │
│                       varchar                        │  double  │       double       │         double         │
├──────────────────────────────────────────────────────┼──────────┼────────────────────┼────────────────────────┤
│ Bhutan                                               │     76.0 │               76.0 │                    1.0 │
│ Qatar                                                │    51.15 │              51.15 │                    1.0 │
│ Nepal                                                │ 265475.0 │ 265486.18000000005 │     0.9999578885801135 │
│ Bosnia and Herzegovina                               │     23.0 │              23.95 │     0.9603340292275574 │
│ Azerbaijan                                           │  3315.69 │ 3459.8500000000004 │